# **Trabajamos con GPT-3.5**

In [14]:
import pandas as pd
import warnings
import json
import demoji
import re
import numpy as np
warnings.filterwarnings("ignore")

In [15]:
# Importar libreria openai para trabajar con su API
import openai
openai.api_key = "AUTH_KEY"

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [16]:
df = pd.read_csv('reviews.csv', sep=';')
#df = df.head(5)
# Crear la columna 'sentiment'
df['sentiment'] = None

In [17]:


def cleaning_text(text):
# TRATAR TRADUCCIONES EN REVIEWS, BORRA EL STRING "(ORIGINAL)" Y TODO A SU DERECHA
  keyword = "(Original)"
  keyword_index = text.find(keyword) # Encontrar la posición de la palabra clave
  if keyword_index != -1:
      text = text[:keyword_index + len(keyword)] # Eliminar todo después de la palabra clave

  text = text.replace("(Original)", "")
  text = text.replace("(Translated by Google)", "")

# TRATAR EMOJIS
  text = demoji.replace_with_desc(text, ' ') # reemplaza los emojis con palabras mas un espacio entre si

# BORRAR SALTOS DE LINEA
  text = text.replace("\n", " ")
# BORRAR CARACTERES NO ALFANUMERICOS
  text = re.sub(r'[^\w\s]', '', text)
# BORRAR ESPACIOS AL PRINCIPIO Y AL FINAL Y CONVERTIR A MINUSCULAS
  text = text.strip()
  text = text.lower()
# DESPUES DE LIMPIADO VERIFICAR SI HAY CARACTERES O NO, SI NO HAY CONVERTIRLOS EN NULO
  if not text:
      return None
  return text

In [18]:
# Crear columna 'cleaned' a partir de la limpieza de la columna 'review'
df["cleaned"] = [cleaning_text(x) for x in df["review"]]

In [24]:
# Obtener el análisis de sentimiento de los reviews y de sus categorías
df['sentiment'] = [
    get_completion(
        f""" 1. Give me the overall sentiment of the next review, the response can be 1 for positive, -1 for negative or 0 for neutral.
            2. Give me the sentiment of the next categories of the review: room, guest service, cleaning and breakfast.
            The response must be 1 for positive, -1 for negative or 0 neutral.
            3. The output must be in JSON format and the keys must be 'overall_sentiment', 'room_sentiment', 'guest_service_sentiment','cleaning_sentiment' and 'breakfast_sentiment'.
            \ ```{review}```
        """
    )
    for review in df['cleaned']
]

In [ ]:

#Convertir cada cadena en la columna 'json_strings' en un objeto JSON
def convert_to_json(row):
    return json.loads(row)

df['sentiment'] = [convert_to_json(x) for x in df["sentiment"]]

In [ ]:
# Crear el 'df_openai' a partir de 'df'
df_openai = df[['review_id','lodging_id','guest_id','date','review','rating','sentiment']]

In [ ]:
# Crear el archivo 'review_openai.csv'
df_openai.to_csv('df_openai.csv',sep=';')

In [144]:
# Crear un dataframe flat llamado 'result_df' desanidando la columna 'sentiment'
# Expandir el diccionario en columnas individuales
expanded_df = pd.DataFrame(df['sentiment'].tolist())

# Combinar los DataFrames originales y expandidos
result_df = pd.concat([df, expanded_df], axis=1)

# Eliminar la columna 'sentiments' si ya no la necesitas
result_df.drop('sentiment', axis=1, inplace=True)

In [145]:
# Crear archivo 'review_openai_flat' a partir de 'result_df'
df_openai_flat = result_df.copy()
df_openai_flat.to_csv('df_openai_flat.csv',sep=';')